<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/Deep-learning-examples-using-keras/blob/main/flower_classification_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -uq "/content/drive/My Drive/kaggle_dataset/flower_data.zip" -d "/content/drive/My Drive/kaggle_dataset/flower_data/"

In [ ]:
train_path = "/content/drive/My Drive/kaggle_dataset/flower_data/flower_data/train"
test_path = "/content/drive/My Drive/kaggle_dataset/flower_data/flower_data/valid"

In [ ]:
import cv2 
im = cv2.imread("/content/drive/My Drive/kaggle_dataset/flower_data/flower_data/train/1/image_06744.jpg")
im.shape

In [ ]:
# Create directories and subdirectories to save resized images
import glob
import os
import shutil

for dir in glob.glob("/content/drive/My Drive/kaggle_dataset/flower_data/flower_data/*/*"):
  split= dir.split("/")
  path_folder = os.path.join("/content/drive/My Drive/kaggle_dataset/flower_data/resized/",split[7], split[8])
  if os.path.exists(path_folder):
    print(path_folder + " exists")
  else:
    os.makedirs(path_folder)

In [ ]:
# Resize training images of 'n' classes
resize_path_train = "/content/drive/My Drive/kaggle_dataset/flower_data/resized/train/"  
for i in range(0, 103):
    for im in glob.glob(os.path.join("/content/drive/My Drive/kaggle_dataset/flower_data/flower_data/train",str(i),'*')):
      img  = cv2.imread(im)
      width = 300
      height = 300
      dim = (width, height)
      # resize image
      resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
      cv2.imwrite(os.path.join(resize_path_train, str(i), im.split("/")[-1]), resized)
      print(os.path.join(resize_path_train, str(i), im.split("/")[-1]))

In [ ]:
# Resize testing images of 'n' classes
resize_path_test = "/content/drive/My Drive/kaggle_dataset/flower_data/resized/valid/"
for i in range(0, 103):
  for im in glob.glob(os.path.join("/content/drive/My Drive/kaggle_dataset/flower_data/flower_data/valid",str(i),'*')):
    img  = cv2.imread(im)
    width = 300
    height = 300
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    cv2.imwrite(os.path.join(resize_path_test, str(i), im.split("/")[-1]), resized)
    print(os.path.join(resize_path_test, str(i), im.split("/")[-1]))
      

In [ ]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/flower_data/resized/train',
                                                 target_size = (224,224),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/flower_data/resized/valid/',
                                            target_size = (224,224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

In [ ]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[224,224, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a third convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a third convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=102, activation='softmax'))

In [ ]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])



In [ ]:
cnn.summary()

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = cnn.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,

)

In [ ]:
cnn.save("flowe_classification_model.h5")
print("Saved model to disk")